In [ ]:
pip install deepctr

     |████████████████████████████████| 112kB 7.7MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/Tobigs/컨퍼런스_와인추천/

Mounted at /content/drive
/content/drive/My Drive/Tobigs/컨퍼런스_와인추천


In [ ]:
import pandas as pd
import numpy as np
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
train_df = pd.read_json('Data 최종본/train_all_meta_v2.json')
test_df = pd.read_json('Data 최종본/test_all_meta_v2.json')

feature : userID, food, grapes, region_id + 연속형 변수 +wine_id



In [ ]:
X_train = train_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','winery_ratings_count','winery_ratings_average','user_follower_count','user_following_count','user_rating_count','userID','wine_id','food','grapes','region_id']]
y_train = train_df['like']
X_test = test_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','winery_ratings_count','winery_ratings_average','user_follower_count','user_following_count','user_rating_count','userID','wine_id','food','grapes','region_id']]
y_test = test_df['like']

In [ ]:
# food 리스트 -> str
X_train['food'] = X_train['food'].fillna(False)  # nan -> False
food_str = []
for i in range(len(X_train['food'])):
    if X_train['food'][i]:food_str.append(' '.join(X_train['food'][i]))
    else:food_str.append(' ')
X_train.food = pd.Series(food_str, name='food')

X_test['food'] = X_test['food'].fillna(False)   # nan -> False
food_str = []
for i in range(len(X_test['food'])):
    if X_test['food'][i]:food_str.append(' '.join(X_test['food'][i]))
    else:food_str.append(' ')
X_test.food = pd.Series(food_str, name='food')

# grapes 리스트 -> str
X_train['grapes'] = X_train['grapes'].fillna(False)   # nan -> False
grapes_str = []
for i in range(len(X_train['grapes'])):
    if X_train['grapes'][i]:grapes_str.append(' '.join(X_train['grapes'][i]))
    else:grapes_str.append(' ')
X_train.grapes = pd.Series(grapes_str, name='grapes')

X_test['grapes'] = X_test['grapes'].fillna(False)   # nan -> False
grapes_str = []
for i in range(len(X_test['grapes'])):
    if X_test['grapes'][i]:grapes_str.append(' '.join(X_test['grapes'][i]))
    else:grapes_str.append(' ')
X_test.grapes = pd.Series(grapes_str, name='grapes')

# region_id -> int
X_train.region_id.fillna(0, inplace=True)
X_test.region_id.fillna(0, inplace=True)
X_train.region_id = X_train.region_id.astype(int)
X_test.region_id = X_test.region_id.astype(int)

In [ ]:
sparse_features = ['userID', 'wine_id', 'food', 'grapes', 'region_id']

dense_features = ['rating_count', 'rating_average', 'body', 'acidity', 'alcohol',
       'winery_ratings_count', 'winery_ratings_average', 'user_follower_count',
       'user_following_count', 'user_rating_count']

for feat in sparse_features:
    lbe = LabelEncoder()
    all = pd.concat([X_train[feat], X_test[feat]], axis=0).drop_duplicates() # train, test 전체를 묶어서 LabelEncoder fit
    lbe = lbe.fit(all)
    X_train[feat] = lbe.transform(X_train[feat])
    X_test[feat] = lbe.transform(X_test[feat])
    if feat == 'wine_id':wine_id_lbe = lbe


mms = MinMaxScaler(feature_range=(0, 1))
X_train[dense_features] = mms.fit_transform(X_train[dense_features])
X_test[dense_features] = mms.transform(X_test[dense_features])

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=pd.concat([X_train[feat], X_test[feat]], axis=0).drop_duplicates().nunique(),embedding_dim=4)  # vocabulary_size를 train, test 전체로부터
                           for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,) for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 결측치 mean으로 처리
X_train.fillna(X_train.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)

train_model_input = {name:X_train[name] for name in feature_names}
test_model_input = {name:X_test[name] for name in feature_names}

In [ ]:
# parameters
BATCH_SIZE = 256
EPOCHS = 10
# early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [ ]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'])  # learning rate는 default 0.001

history = model.fit(train_model_input, y_train.values,
                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=2, validation_split=0.2)
y_pred = model.predict(test_model_input, batch_size=BATCH_SIZE)
print("test AUC", round(roc_auc_score(y_test.values, y_pred), 4))
print("\ntest Auccuracy", round(accuracy_score(y_test.values, y_pred.round()), 4))

Epoch 1/10
2386/2386 - 27s - loss: 0.5704 - binary_crossentropy: 0.5684 - val_loss: 0.6546 - val_binary_crossentropy: 0.6512
Epoch 2/10
2386/2386 - 24s - loss: 0.5352 - binary_crossentropy: 0.5294 - val_loss: 0.6651 - val_binary_crossentropy: 0.6580
Epoch 3/10
2386/2386 - 24s - loss: 0.5199 - binary_crossentropy: 0.5108 - val_loss: 0.6658 - val_binary_crossentropy: 0.6555
Epoch 4/10
2386/2386 - 24s - loss: 0.5041 - binary_crossentropy: 0.4921 - val_loss: 0.6837 - val_binary_crossentropy: 0.6706
Epoch 5/10
2386/2386 - 24s - loss: 0.4903 - binary_crossentropy: 0.4755 - val_loss: 0.7046 - val_binary_crossentropy: 0.6888
Epoch 6/10
2386/2386 - 24s - loss: 0.4755 - binary_crossentropy: 0.4584 - val_loss: 0.7193 - val_binary_crossentropy: 0.7011
Epoch 7/10
2386/2386 - 23s - loss: 0.4607 - binary_crossentropy: 0.4414 - val_loss: 0.7158 - val_binary_crossentropy: 0.6955
Epoch 8/10
2386/2386 - 24s - loss: 0.4480 - binary_crossentropy: 0.4269 - val_loss: 0.7434 - val_binary_crossentropy: 0.7215


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
userID (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
wine_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
food (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
grapes (InputLayer)             [(None, 1)]          0                                            
______________________________________________________________________________________________

feature : userID, wine_id, food, grapes, region_id, country_code, 연속형 변수

In [ ]:
X_train = train_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','winery_id','userID','wine_id','food','grapes','region_id','country_code']]
y_train = train_df['like']
X_test = test_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','winery_id','userID','wine_id','food','grapes','region_id','country_code']]
y_test = test_df['like']

In [ ]:
# food 리스트 -> str
food_str = []
for i in range(len(X_train['food'])):
    if X_train['food'][i]:food_str.append(' '.join(X_train['food'][i]))
    else:food_str.append(' ')
X_train.food = pd.Series(food_str, name='food')

food_str = []
for i in range(len(X_test['food'])):
    if X_test['food'][i]:food_str.append(' '.join(X_test['food'][i]))
    else:food_str.append(' ')
X_test.food = pd.Series(food_str, name='food')

# grapes 리스트 -> str
grapes_str = []
for i in range(len(X_train['grapes'])):
    if X_train['grapes'][i]:grapes_str.append(' '.join(X_train['grapes'][i]))
    else:grapes_str.append(' ')
X_train.grapes = pd.Series(grapes_str, name='grapes')

grapes_str = []
for i in range(len(X_test['grapes'])):
    if X_test['grapes'][i]:grapes_str.append(' '.join(X_test['grapes'][i]))
    else:grapes_str.append(' ')
X_test.grapes = pd.Series(grapes_str, name='grapes')

# region_id -> int
X_train.region_id.fillna(0, inplace=True)
X_test.region_id.fillna(0, inplace=True)
X_train.region_id = X_train.region_id.astype(int)
X_test.region_id = X_test.region_id.astype(int)

# winery_id -> int
X_train.winery_id.fillna(0, inplace=True)
X_test.winery_id.fillna(0, inplace=True)
X_train.winery_id = X_train.winery_id.astype(int)
X_test.winery_id = X_test.winery_id.astype(int)

# country_code
X_train.country_code.fillna('un', inplace=True)
X_test.country_code.fillna('un', inplace=True)

In [ ]:
sparse_features = ['userID', 'wine_id', 'food', 'grapes', 'region_id', 'country_code', 'winery_id']

dense_features = ['rating_count', 'rating_average', 'body', 'acidity', 'alcohol']

for feat in sparse_features:
    lbe = LabelEncoder()
    all = pd.concat([X_train[feat], X_test[feat]], axis=0).drop_duplicates() # train, test 전체를 묶어서 LabelEncoder fit
    lbe = lbe.fit(all)
    X_train[feat] = lbe.transform(X_train[feat])
    X_test[feat] = lbe.transform(X_test[feat])
    if feat == 'wine_id':wine_id_lbe = lbe


mms = MinMaxScaler(feature_range=(0, 1))
X_train[dense_features] = mms.fit_transform(X_train[dense_features])
X_test[dense_features] = mms.transform(X_test[dense_features])

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=pd.concat([X_train[feat], X_test[feat]], axis=0).drop_duplicates().nunique(),embedding_dim=EMBEDDING_DIM)  # vocabulary_size를 train, test 전체로부터
                           for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,) for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 결측치 mean으로 처리
X_train.fillna(X_train.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)

train_model_input = {name:X_train[name] for name in feature_names}
test_model_input = {name:X_test[name] for name in feature_names}

In [ ]:
# parameters
BATCH_SIZE = 256
EPOCHS = 100
EMBEDDING_DIM = 20
DROPOUT_RATE = 0.25
early_stopping = EarlyStopping(monitor='val_loss', verbose=1, patience=10)

In [ ]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary', dnn_dropout=DROPOUT_RATE, dnn_use_bn=True, l2_reg_embedding=0.01)
model.compile("adam", "binary_crossentropy",metrics=['accuracy'])  # learning rate는 default 0.001

history = model.fit(train_model_input, y_train.values,
                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=2, validation_split=0.2, callbacks=[early_stopping])  # 
y_pred = model.predict(test_model_input, batch_size=BATCH_SIZE)
print("test AUC", round(roc_auc_score(y_test.values, y_pred), 4))
print("\ntest Auccuracy", round(accuracy_score(y_test.values, y_pred.round()), 4))

Epoch 1/200
2386/2386 - 46s - loss: 0.6612 - accuracy: 0.6992 - val_loss: 0.7034 - val_accuracy: 0.6531
Epoch 2/200
2386/2386 - 44s - loss: 0.6187 - accuracy: 0.7173 - val_loss: 0.6954 - val_accuracy: 0.6554
Epoch 3/200
2386/2386 - 44s - loss: 0.6086 - accuracy: 0.7229 - val_loss: 0.7069 - val_accuracy: 0.6565
Epoch 4/200
2386/2386 - 44s - loss: 0.6049 - accuracy: 0.7251 - val_loss: 0.6983 - val_accuracy: 0.6496
Epoch 5/200
2386/2386 - 44s - loss: 0.6016 - accuracy: 0.7271 - val_loss: 0.6968 - val_accuracy: 0.6554
Epoch 6/200
2386/2386 - 44s - loss: 0.5998 - accuracy: 0.7282 - val_loss: 0.7001 - val_accuracy: 0.6537
Epoch 7/200
2386/2386 - 44s - loss: 0.5962 - accuracy: 0.7293 - val_loss: 0.6913 - val_accuracy: 0.6517
Epoch 8/200
2386/2386 - 44s - loss: 0.5962 - accuracy: 0.7296 - val_loss: 0.6898 - val_accuracy: 0.6537
Epoch 9/200
2386/2386 - 44s - loss: 0.5941 - accuracy: 0.7305 - val_loss: 0.6923 - val_accuracy: 0.6549
Epoch 10/200
2386/2386 - 44s - loss: 0.5929 - accuracy: 0.7309 -

In [ ]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary', dnn_dropout=DROPOUT_RATE, dnn_use_bn=True, l2_reg_linear=0.0001, l2_reg_embedding=0.0001)
model.compile("adam", "binary_crossentropy",metrics=['accuracy'])  # learning rate는 default 0.001

history = model.fit(train_model_input, y_train.values,
                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=2, validation_split=0.2, callbacks=[early_stopping])  # 
y_pred = model.predict(test_model_input, batch_size=BATCH_SIZE)
print("test AUC", round(roc_auc_score(y_test.values, y_pred), 4))
print("\ntest Auccuracy", round(accuracy_score(y_test.values, y_pred.round()), 4))

Epoch 1/200
2386/2386 - 47s - loss: 0.5940 - accuracy: 0.7079 - val_loss: 0.6805 - val_accuracy: 0.6476
Epoch 2/200
2386/2386 - 44s - loss: 0.5716 - accuracy: 0.7268 - val_loss: 0.6726 - val_accuracy: 0.6509
Epoch 3/200
2386/2386 - 45s - loss: 0.5684 - accuracy: 0.7307 - val_loss: 0.6616 - val_accuracy: 0.6527
Epoch 4/200
2386/2386 - 45s - loss: 0.5660 - accuracy: 0.7335 - val_loss: 0.6710 - val_accuracy: 0.6546
Epoch 5/200
2386/2386 - 45s - loss: 0.5640 - accuracy: 0.7371 - val_loss: 0.6756 - val_accuracy: 0.6533
Epoch 6/200
2386/2386 - 44s - loss: 0.5607 - accuracy: 0.7416 - val_loss: 0.6780 - val_accuracy: 0.6524
Epoch 7/200
2386/2386 - 45s - loss: 0.5573 - accuracy: 0.7471 - val_loss: 0.6891 - val_accuracy: 0.6498
Epoch 8/200
2386/2386 - 45s - loss: 0.5545 - accuracy: 0.7518 - val_loss: 0.6961 - val_accuracy: 0.6488
Epoch 9/200
2386/2386 - 44s - loss: 0.5514 - accuracy: 0.7559 - val_loss: 0.6995 - val_accuracy: 0.6470
Epoch 10/200
2386/2386 - 44s - loss: 0.5484 - accuracy: 0.7591 -

In [ ]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary', dnn_dropout=DROPOUT_RATE, dnn_use_bn=True)
model.compile("adam", "binary_crossentropy",metrics=['accuracy'])  # learning rate는 default 0.001

history = model.fit(train_model_input, y_train.values,
                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=2, validation_split=0.2)  # , callbacks=[early_stopping]
y_pred = model.predict(test_model_input, batch_size=BATCH_SIZE)
print("test AUC", round(roc_auc_score(y_test.values, y_pred), 4))
print("\ntest Auccuracy", round(accuracy_score(y_test.values, y_pred.round()), 4))

Epoch 1/100
2386/2386 - 46s - loss: 0.5738 - accuracy: 0.7069 - val_loss: 0.6856 - val_accuracy: 0.6439
Epoch 2/100
2386/2386 - 44s - loss: 0.5426 - accuracy: 0.7343 - val_loss: 0.6799 - val_accuracy: 0.6509
Epoch 3/100
2386/2386 - 44s - loss: 0.5323 - accuracy: 0.7469 - val_loss: 0.6846 - val_accuracy: 0.6496
Epoch 4/100
2386/2386 - 44s - loss: 0.5147 - accuracy: 0.7650 - val_loss: 0.6915 - val_accuracy: 0.6454
Epoch 5/100
2386/2386 - 43s - loss: 0.4852 - accuracy: 0.7888 - val_loss: 0.7163 - val_accuracy: 0.6415
Epoch 6/100
2386/2386 - 44s - loss: 0.4513 - accuracy: 0.8111 - val_loss: 0.7421 - val_accuracy: 0.6365
Epoch 7/100
2386/2386 - 43s - loss: 0.4240 - accuracy: 0.8273 - val_loss: 0.7690 - val_accuracy: 0.6311
Epoch 8/100
2386/2386 - 43s - loss: 0.4039 - accuracy: 0.8389 - val_loss: 0.7713 - val_accuracy: 0.6281
Epoch 9/100
2386/2386 - 43s - loss: 0.3893 - accuracy: 0.8470 - val_loss: 0.8051 - val_accuracy: 0.6268
Epoch 10/100
2386/2386 - 43s - loss: 0.3792 - accuracy: 0.8521 -

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
userID (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
wine_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
food (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
grapes (InputLayer)             [(None, 1)]          0                                            
______________________________________________________________________________________________

dropout, batch normalization 안하고 100번 돌릴 경우

In [ ]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy",metrics=['accuracy'])  # learning rate는 default 0.001

history = model.fit(train_model_input, y_train.values,
                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=2, validation_split=0.2)  # , callbacks=[early_stopping]
y_pred = model.predict(test_model_input, batch_size=BATCH_SIZE)
print("test AUC", round(roc_auc_score(y_test.values, y_pred), 4))
print("\ntest Auccuracy", round(accuracy_score(y_test.values, y_pred.round()), 4))

Epoch 1/100
2386/2386 - 48s - loss: 0.5805 - accuracy: 0.6989 - val_loss: 0.6641 - val_accuracy: 0.6486
Epoch 2/100
2386/2386 - 44s - loss: 0.5472 - accuracy: 0.7319 - val_loss: 0.6657 - val_accuracy: 0.6509
Epoch 3/100
2386/2386 - 43s - loss: 0.5331 - accuracy: 0.7453 - val_loss: 0.6794 - val_accuracy: 0.6485
Epoch 4/100
2386/2386 - 42s - loss: 0.5171 - accuracy: 0.7594 - val_loss: 0.6937 - val_accuracy: 0.6481
Epoch 5/100
2386/2386 - 44s - loss: 0.4953 - accuracy: 0.7772 - val_loss: 0.7161 - val_accuracy: 0.6384
Epoch 6/100
2386/2386 - 45s - loss: 0.4639 - accuracy: 0.7998 - val_loss: 0.7283 - val_accuracy: 0.6387
Epoch 7/100
2386/2386 - 43s - loss: 0.4248 - accuracy: 0.8245 - val_loss: 0.7832 - val_accuracy: 0.6345
Epoch 8/100
2386/2386 - 42s - loss: 0.3870 - accuracy: 0.8459 - val_loss: 0.8108 - val_accuracy: 0.6330
Epoch 9/100
2386/2386 - 41s - loss: 0.3557 - accuracy: 0.8623 - val_loss: 0.8618 - val_accuracy: 0.6319
Epoch 10/100
2386/2386 - 43s - loss: 0.3349 - accuracy: 0.8730 -

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
userID (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
wine_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
food (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
grapes (InputLayer)             [(None, 1)]          0                                            
____________________________________________________________________________________________

embedding_dim 32로 늘리고 early_stopping train accuracy로 걸어두는 경우 **(dropout & batchnormalization O)**

In [ ]:
# parameters
BATCH_SIZE = 256
EPOCHS = 500
EMBEDDING_DIM = 32
DROPOUT_RATE = 0.25
early_stopping = EarlyStopping(monitor='accuracy', verbose=1, patience=10)

In [ ]:
sparse_features = ['userID', 'wine_id', 'food', 'grapes', 'region_id', 'country_code', 'winery_id']

dense_features = ['rating_count', 'rating_average', 'body', 'acidity', 'alcohol']

for feat in sparse_features:
    lbe = LabelEncoder()
    all = pd.concat([X_train[feat], X_test[feat]], axis=0).drop_duplicates() # train, test 전체를 묶어서 LabelEncoder fit
    lbe = lbe.fit(all)
    X_train[feat] = lbe.transform(X_train[feat])
    X_test[feat] = lbe.transform(X_test[feat])
    if feat == 'wine_id':wine_id_lbe = lbe
    if feat == 'food':food_lbe = lbe
    if feat == 'grapes':grapes_lbe = lbe


mms = MinMaxScaler(feature_range=(0, 1))
X_train[dense_features] = mms.fit_transform(X_train[dense_features])
X_test[dense_features] = mms.transform(X_test[dense_features])

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=pd.concat([X_train[feat], X_test[feat]], axis=0).drop_duplicates().nunique(),embedding_dim=EMBEDDING_DIM)  # vocabulary_size를 train, test 전체로부터
                           for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,) for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 결측치 mean으로 처리
X_train.fillna(X_train.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)

train_model_input = {name:X_train[name] for name in feature_names}
test_model_input = {name:X_test[name] for name in feature_names}

In [ ]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary', dnn_dropout=DROPOUT_RATE, dnn_use_bn=True)
model.compile("adam", "binary_crossentropy",metrics=['accuracy'])  # learning rate는 default 0.001

history = model.fit(train_model_input, y_train.values,
                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=2, validation_split=0.2, callbacks=[early_stopping])  # 
y_pred = model.predict(test_model_input, batch_size=BATCH_SIZE)
print("test AUC", round(roc_auc_score(y_test.values, y_pred), 4))
print("\ntest Auccuracy", round(accuracy_score(y_test.values, y_pred.round()), 4))

Epoch 1/500
2386/2386 - 62s - loss: 0.5777 - accuracy: 0.7073 - val_loss: 0.6829 - val_accuracy: 0.6412
Epoch 2/500
2386/2386 - 58s - loss: 0.5487 - accuracy: 0.7336 - val_loss: 0.6790 - val_accuracy: 0.6515
Epoch 3/500
2386/2386 - 58s - loss: 0.5405 - accuracy: 0.7462 - val_loss: 0.6820 - val_accuracy: 0.6501
Epoch 4/500
2386/2386 - 57s - loss: 0.5260 - accuracy: 0.7632 - val_loss: 0.7028 - val_accuracy: 0.6487
Epoch 5/500
2386/2386 - 57s - loss: 0.4982 - accuracy: 0.7870 - val_loss: 0.7192 - val_accuracy: 0.6427
Epoch 6/500
2386/2386 - 57s - loss: 0.4645 - accuracy: 0.8110 - val_loss: 0.7469 - val_accuracy: 0.6372
Epoch 7/500
2386/2386 - 57s - loss: 0.4356 - accuracy: 0.8295 - val_loss: 0.7804 - val_accuracy: 0.6294
Epoch 8/500
2386/2386 - 57s - loss: 0.4146 - accuracy: 0.8412 - val_loss: 0.7875 - val_accuracy: 0.6273
Epoch 9/500
2386/2386 - 56s - loss: 0.3995 - accuracy: 0.8493 - val_loss: 0.7938 - val_accuracy: 0.6231
Epoch 10/500
2386/2386 - 55s - loss: 0.3892 - accuracy: 0.8560 -

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
userID (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
wine_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
food (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
grapes (InputLayer)             [(None, 1)]          0                                            
______________________________________________________________________________________________

embedding_dim 32로 늘리고 early_stopping train accuracy로 걸어두는 경우 **(dropout & batchnormalization X)**

In [ ]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy",metrics=['accuracy'])  # learning rate는 default 0.001

history = model.fit(train_model_input, y_train.values,
                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=2, validation_split=0.2, callbacks=[early_stopping])  # 
y_pred = model.predict(test_model_input, batch_size=BATCH_SIZE)
print("test AUC", round(roc_auc_score(y_test.values, y_pred), 4))
print("\ntest Auccuracy", round(accuracy_score(y_test.values, y_pred.round()), 4))

Epoch 1/500
2386/2386 - 70s - loss: 0.5858 - accuracy: 0.6961 - val_loss: 0.6609 - val_accuracy: 0.6509
Epoch 2/500
2386/2386 - 65s - loss: 0.5528 - accuracy: 0.7316 - val_loss: 0.6640 - val_accuracy: 0.6507
Epoch 3/500
2386/2386 - 71s - loss: 0.5401 - accuracy: 0.7450 - val_loss: 0.6874 - val_accuracy: 0.6447
Epoch 4/500
2386/2386 - 79s - loss: 0.5248 - accuracy: 0.7587 - val_loss: 0.6971 - val_accuracy: 0.6433
Epoch 5/500
2386/2386 - 71s - loss: 0.5039 - accuracy: 0.7761 - val_loss: 0.7116 - val_accuracy: 0.6422
Epoch 6/500
2386/2386 - 73s - loss: 0.4743 - accuracy: 0.7979 - val_loss: 0.7340 - val_accuracy: 0.6377
Epoch 7/500
2386/2386 - 68s - loss: 0.4368 - accuracy: 0.8222 - val_loss: 0.7795 - val_accuracy: 0.6349
Epoch 8/500
2386/2386 - 64s - loss: 0.3957 - accuracy: 0.8472 - val_loss: 0.8576 - val_accuracy: 0.6308
Epoch 9/500
2386/2386 - 62s - loss: 0.3625 - accuracy: 0.8651 - val_loss: 0.9096 - val_accuracy: 0.6292
Epoch 10/500
2386/2386 - 62s - loss: 0.3391 - accuracy: 0.8781 -

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
userID (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
wine_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
food (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
grapes (InputLayer)             [(None, 1)]          0                                            
____________________________________________________________________________________________

#### 최선의 모델의 embedding으로 새로운 유저에게 추천해보기

In [ ]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary', dnn_dropout=DROPOUT_RATE, dnn_use_bn=True)
model.compile("adam", "binary_crossentropy",metrics=['accuracy'])  # learning rate는 default 0.001

history = model.fit(train_model_input, y_train.values,
                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=0, validation_split=0.2, callbacks=[early_stopping])  # 
y_pred = model.predict(test_model_input, batch_size=BATCH_SIZE)
print("test AUC", round(roc_auc_score(y_test.values, y_pred), 4))
print("\ntest Auccuracy", round(accuracy_score(y_test.values, y_pred.round()), 4))

model.save_weights('DeepFM_20210101.h5')
print("model saved!")

Epoch 00228: early stopping
test AUC 0.9246

test Auccuracy 0.8462
model saved!


In [ ]:
X_train = train_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','winery_id','userID','wine_id','food','grapes','region_id','country_code']]
y_train = train_df['like']
X_test = test_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','winery_id','userID','wine_id','food','grapes','region_id','country_code']]
y_test = test_df['like']

In [ ]:
# food 리스트 -> str
food_str = []
for i in range(len(X_train['food'])):
    if X_train['food'][i]:food_str.append('/'.join(X_train['food'][i]))  # 구분자 '/'로 변경
    else:food_str.append('')
X_train.food = pd.Series(food_str, name='food')

food_str = []
for i in range(len(X_test['food'])):
    if X_test['food'][i]:food_str.append('/'.join(X_test['food'][i]))
    else:food_str.append('')
X_test.food = pd.Series(food_str, name='food')

# grapes 리스트 -> str
grapes_str = []
for i in range(len(X_train['grapes'])):
    if X_train['grapes'][i]:grapes_str.append('/'.join(X_train['grapes'][i]))
    else:grapes_str.append('')
X_train.grapes = pd.Series(grapes_str, name='grapes')

grapes_str = []
for i in range(len(X_test['grapes'])):
    if X_test['grapes'][i]:grapes_str.append('/'.join(X_test['grapes'][i]))
    else:grapes_str.append('')
X_test.grapes = pd.Series(grapes_str, name='grapes')

# region_id -> int
X_train.region_id.fillna(0, inplace=True)
X_test.region_id.fillna(0, inplace=True)
X_train.region_id = X_train.region_id.astype(int)
X_test.region_id = X_test.region_id.astype(int)

# winery_id -> int
X_train.winery_id.fillna(0, inplace=True)
X_test.winery_id.fillna(0, inplace=True)
X_train.winery_id = X_train.winery_id.astype(int)
X_test.winery_id = X_test.winery_id.astype(int)

# country_code
X_train.country_code.fillna('un', inplace=True)
X_test.country_code.fillna('un', inplace=True)

In [ ]:
EMBEDDING_DIM = 32
DROPOUT_RATE = 0.25

In [ ]:
sparse_features = ['userID', 'wine_id', 'food', 'grapes', 'region_id', 'country_code', 'winery_id']

dense_features = ['rating_count', 'rating_average', 'body', 'acidity', 'alcohol']

for feat in sparse_features:
    lbe = LabelEncoder()
    all = pd.concat([X_train[feat], X_test[feat]], axis=0).drop_duplicates() # train, test 전체를 묶어서 LabelEncoder fit
    lbe = lbe.fit(all)
    X_train[feat] = lbe.transform(X_train[feat])
    X_test[feat] = lbe.transform(X_test[feat])
    if feat == 'wine_id':wine_id_lbe = lbe
    if feat == 'food':food_lbe = lbe
    if feat == 'grapes':grapes_lbe = lbe

# for feat in dense_features:
#     mms = MinMaxScaler(feature_range=(0, 1))
#     X_train[feat] = mms.fit_transform(X_train[feat].values.reshape(-1,1))
#     X_test[feat] = mms.transform(X_test[feat].values.reshape(-1,1))
#     if feat == 'body':body_mms = mms
#     if feat == 'acidity':acidity_mms = mms
#     if feat == 'alcohol':alcohol_mms = mms

mms = MinMaxScaler(feature_range=(0, 1))
X_train[dense_features] = mms.fit_transform(X_train[dense_features])
X_test[dense_features] = mms.transform(X_test[dense_features])

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=pd.concat([X_train[feat], X_test[feat]], axis=0).drop_duplicates().nunique(),embedding_dim=EMBEDDING_DIM)  # vocabulary_size를 train, test 전체로부터
                           for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,) for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 결측치 mean으로 처리
X_train.fillna(X_train.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)

In [ ]:
model_load = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary', dnn_dropout=DROPOUT_RATE, dnn_use_bn=True)
model_load.load_weights('DeepFM_20210101.h5')

In [ ]:
# model embedding
def get_embedding_weights(dnn_feature_columns,model):
    embedding_dict = {}
    for fc in dnn_feature_columns:
        if hasattr(fc,'embedding_name'):
            if fc.embedding_name is not None:
                name = fc.embedding_name
            else:
                name = fc.name
            embedding_dict[name] = model.get_layer("sparse_emb_"+name).get_weights()[0]
    return embedding_dict
    
embedding_dict = get_embedding_weights(fixlen_feature_columns, model_load)

wine_id_emb = embedding_dict['wine_id']
food_emb = embedding_dict['food']

In [ ]:
food_list_label = pd.concat([train_df.food, X_train.food], axis=1, keys=['list', 'label']).drop_duplicates(['label']).reset_index(drop=True)
food_all = pd.concat([food_list_label, pd.Series(list(food_emb[food_list_label.label]), name='emb')], axis=1)
food_all

,list,label,emb
0,"[Pork, Rich fish (salmon, tuna etc), Shellfish...",269,"[0.05657315, 0.021117035, -0.062607296, -0.024..."
1,"[Beef, Lamb, Game (deer, venison), Spicy food,...",25,"[0.015593277, -0.00957169, 0.011200247, -0.027..."
2,"[Beef, Lamb, Spicy food, Mature and hard cheese]",94,"[0.0057849647, -0.041216485, 0.035466716, -0.0..."
3,"[Pork, Rich fish (salmon, tuna etc), Spicy foo...",296,"[-0.0346549, -0.04446286, -0.020822234, -0.024..."
4,"[Shellfish, Vegetarian, Appetizers and snacks,...",355,"[-0.05230138, -0.031919636, -0.087989956, 0.01..."
...,...,...,...
393,"[Beef, Lamb, Spicy food, Goat cheese]",93,"[-9.764262e-05, -0.010250316, -0.0017807842, -..."
394,"[Beef, Pork, Game (deer, venison), Rich fish (...",160,"[0.00725575, 0.002483638, -0.0016022181, 0.001..."
395,"[Beef, Lamb, Rich fish (salmon, tuna etc), She...",89,"[0.005134615, 0.007591334, -0.011012361, -0.00..."
396,"[Beef, Lamb, Veal, Pork, Rich fish (salmon, tu...",136,"[-2.5239377e-12, -2.6888125e-09, 1.3839616e-09..."


In [ ]:
wine_id_label = pd.concat([pd.concat([train_df.wine_id, test_df.wine_id], axis=0), pd.concat([X_train.wine_id, X_test.wine_id], axis=0)], axis=1, keys=['wine_id', 'label']).drop_duplicates(['wine_id']).reset_index(drop=True)
wine_id_all = pd.concat([wine_id_label, pd.Series(list(wine_id_emb[wine_id_label.label]), name='emb')], axis=1)
wine_id_all

,wine_id,label,emb
0,1141133,11762,"[-0.038339917, -0.04009262, -0.10686964, -0.07..."
1,2532733,41415,"[-0.02701318, -0.08218294, 0.1706958, -0.03078..."
2,1253802,22307,"[0.06297077, 0.035630755, 0.029052224, -0.0318..."
3,1123441,9651,"[-0.009040968, -0.0541438, -0.009438185, -0.00..."
4,1157656,13547,"[-0.124105886, 0.09853748, 0.13277, -0.0006996..."
...,...,...,...
50850,1169358,14786,"[-3.2264017e-31, -8.973177e-32, -5.043501e-31,..."
50851,1441950,26808,"[-1.6112924e-31, -4.130354e-31, -4.5249555e-31..."
50852,2688979,42386,"[-4.3398255e-31, 5.6357704e-31, 2.3499355e-31,..."
50853,74720,4953,"[-4.552277e-31, -2.1349984e-31, 3.4143536e-31,..."


In [ ]:
# recommend_new_input에 추가로 들어가는 parameter : food_all, wine_id_all, wine_id_emb, body_mms, acidity_mms, alcohol_mms, X_train, X_test, wine_id_lbe

def recommend_new_input(model, input_food, input_body, input_acidity, input_alcohol, K):
    
    # food_all에서 input_food와 가장 유사한 것 jaccard similarity로 추출
    test_food = set(input_food)
    all_food = [set(food) for food in food_all.list]
    union = [test_food.union(i) for i in all_food]
    intersection = [test_food.intersection(i) for i in all_food]
    jaccard = [len(inter)/len(uni) for inter,uni in zip(intersection, union)]
    input_food_emb = food_all.emb[np.array(jaccard).argsort()[0]]

    # food embedding을 wine_id embedding과 dot product해서 score 산출
    score_by_food = pd.concat([wine_id_all[['wine_id', 'label']], pd.Series(np.dot(input_food_emb, wine_id_emb[wine_id_all.label,:].T), name='score_by_food')], axis=1)

    # body, acidity, alcohol은 모든 wine의 body, acidity, alcohol과 dot product해서 score 산출
    input_body = (input_body-train_df.body.min())/(train_df.body.max()-train_df.body.min())
    input_acidity = (input_acidity-train_df.acidity.min())/(train_df.acidity.max()-train_df.acidity.min())
    input_alcohol = (input_alcohol-train_df.alcohol.min())/(train_df.alcohol.max()-train_df.alcohol.min())
    input_baa = [input_body, input_acidity, input_alcohol]

    all_baa = pd.concat([X_train, X_test], axis=0)[['wine_id', 'body', 'acidity', 'alcohol']].drop_duplicates('wine_id')
    all_baa.drop(['wine_id'], axis=1, inplace=True)
    all_baa = np.array(all_baa)
    score_by_baa = pd.Series(np.dot(input_baa, all_baa.T), name='score_by_baa')

    score_all = pd.concat([score_by_food, score_by_baa], axis=1)
    score_all['final'] = score_all.score_by_food + score_by_baa
    score_all = score_all.sort_values(['final'], ascending=False)[:K]

    return score_all

In [ ]:
food_list_all = []
for food in food_list:food_list_all.extend(food)
food_list_all = pd.Series(food_list_all).value_counts().index[:-1]

print(food_list_all)
input_food = input('다음 중 와인과 함께 곁들여 먹을 음식을 입력하세요 : ').split(',')
input_body = int(input('원하는 바디감을 1,2,3,4,5 중 하나로 입력하세요 : '))
input_acidity = int(input('원하는 산도를 1,2,3 중 하나로 입력하세요 : '))
input_alcohol = int(input('원하는 도수를 5,7,9,11,13 중 하나로 입력하세요 : '))

recommend_new_input(model_load, input_food, input_body, input_acidity, input_alcohol, 10)

Index(['Poultry', 'Pork', 'Beef', 'Rich fish (salmon, tuna etc)', 'Spicy food',
       'Shellfish', 'Game (deer, venison)', 'Mature and hard cheese', 'Lamb',
       'Vegetarian', 'Mild and soft cheese', 'Veal', 'Sweet desserts',
       'Lean fish', 'Mushrooms', 'Cured Meat', 'Goat cheese',
       'Fruity desserts', 'Blue cheese', 'Appetizers and snacks', 'Pasta',
       'Aperitif'],
      dtype='object')
다음 중 와인과 함께 곁들여 먹을 음식을 입력하세요 : Beef, Lamb
원하는 바디감을 1,2,3,4,5 중 하나로 입력하세요 : 4
원하는 산도를 1,2,3 중 하나로 입력하세요 : 2
원하는 도수를 5,7,9,11,13 중 하나로 입력하세요 : 11


,wine_id,label,score_by_food,score_by_baa,final
9649,6162,1187,0.062362,1.259355,1.321717
5114,79377,5615,0.060777,1.259053,1.319831
8759,86247,6579,0.060463,1.258993,1.319456
1174,66164,4578,0.059712,1.258450,1.318161
4573,23109,3748,0.056345,1.257846,1.314191
8536,1881482,35238,0.055274,1.258752,1.314026
3893,1147563,12521,0.051940,1.262071,1.314012
34427,1434561,26669,0.053268,1.258450,1.311718
19303,1084066,8371,0.052664,1.258752,1.311415
1897,6618,1273,0.051704,1.258812,1.310516


Hit rate

In [ ]:
test_all = pd.concat([X_test, y_test], axis=1)
test_all = test_all[test_all.like==1]
hit_user_meta = test_all[['userID','user_follower_count','user_following_count','user_rating_count']]
hit_wine_true = test_all['wine_id']

In [ ]:
wine_meta = pd.concat([X_train, X_test], axis=0)[['wine_id','rating_count','rating_average','body','acidity','alcohol','winery_ratings_count','winery_ratings_average','food','grapes','region_id']].drop_duplicates(['wine_id']).reset_index(drop=True)

In [ ]:
K = 5000
i, hit = 0, 0
wine_id = pd.Series(wine_id_lbe.inverse_transform(wine_meta.wine_id), name='wine_id')

for row, true_wine in zip(hit_user_meta.values, hit_wine_true):

    tmp_df = pd.DataFrame(itertools.repeat(row, len(wine_meta)), columns=hit_user_meta.columns)
    tmp_all = pd.concat([tmp_df, wine_meta], axis=1)
    tmp_input = {name:tmp_all[name] for name in feature_names}

    y_pred = model.predict(tmp_input, batch_size=BATCH_SIZE)
    y_pred = pd.Series(y_pred.T[0], name='y_pred')
    input_predict = pd.concat([wine_id, y_pred], axis=1)
    input_predict = input_predict.sort_values(by='y_pred', ascending=False)
    predict_top_k = input_predict.wine_id[:K].values

    if true_wine in predict_top_k:hit+=1

    i += 1

hit_rate = hit/len(hit_user_meta)

In [ ]:
hit_rate

0.007063112326270221